In [16]:
# Modules
import pandas as pd
import numpy as np
from patsy import dmatrices
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
limma = importr('limma')
# Working directory
wd = "~/Dropbox/Work/Janet Snell-Bergeon/EDIC/Lipidomics/"

In [59]:
# Import raw data
intensities = pd.read_csv(wd+"Data_Cleaned/lipidomics_data_matrix.csv")
samples = pd.read_csv(wd+"Data_Cleaned/study_info.csv")
# Correct sample IDs
samples['sample_id'] = ['EDIC-'+str(i).zfill(3) for i in samples['ProteomicsSampleID']]
samples.index = samples['sample_id'].astype('str')
# Transpose intensities, reindex (strip spaces from index otherwise merge won't work)
intensities = intensities.transpose()
intensities = intensities.rename(columns=intensities.iloc[0,:])
intensities = intensities.drop('LipidSpecies')
intensities = intensities.rename_axis('sample_id')
intensities.index = intensities.index.str.strip()
# Merge
data = samples.merge(intensities,how='left',left_index=True,right_index=True)

In [61]:
pandas2ri.activate()
# Outcomes (lipids)
y = intensities.loc[samples['sample_id']].astype('float').transpose()
#y = y['AcCa(10:0)'].transpose()
# Design matrix - OBS is just a placeholder outcome, we don't actually need it
X = dmatrices('OBS ~ CARV', data=samples, return_type='dataframe')[1]
fit = limma.lmFit(object = y,design = X)
fit = limma.eBayes(fit)
limma.topTable(fit,coef = 2)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
ChE(19:2),-8.027883e+06,1.462389e+08,-0.564890,0.572629,0.999226,-4.59512
DG(18:1_18:1),8.790318e+07,8.021579e+08,1.045336,0.296825,0.999226,-4.59512
LPC(22:5) (3),-3.725842e+06,3.494825e+07,-1.397098,0.163559,0.999226,-4.59512
PC(16:1_20:4),4.206492e+08,2.809063e+09,1.649418,0.100253,0.999226,-4.59512
SM(t42:1),-2.130193e+06,2.085349e+07,-0.995225,0.320540,0.999226,-4.59512
TG(16:0_20:5_22:6),1.569880e+07,6.249242e+07,1.364230,0.173659,0.999226,-4.59512
TG(18:0_18:0_18:1),9.770218e+06,9.548890e+07,0.783056,0.434297,0.999226,-4.59512
TG(18:3_18:2_22:6),1.525514e+07,6.248252e+07,1.327827,0.185384,0.999226,-4.59512
Hex3Cer(d18:1_16:0),-2.645387e+05,2.806081e+06,-0.768359,0.443000,0.999226,-4.59512
AcCa(10:0),6.466890e+06,1.336310e+08,0.530174,0.596438,0.999226,-4.59512
